In [1]:
import os
import sys
import csv
import time
import argparse
import fnmatch
import librosa
import pandas as pd

from hparams import HParams as hp
from zipfile import ZipFile
from audio import preprocess
from utils import download_file
from datasets.tsp_speech import TSPSpeech
from datasets.mb_speech import MBSpeech
from datasets.lj_speech import LJSpeech

In [2]:
import sys
import time
import argparse
from tqdm import *

import numpy as np

import torch
import torch.nn.functional as F

# project imports
from models import Text2Mel
from hparams import HParams as hp
from logger import Logger
from utils import get_last_checkpoint_file_name, load_checkpoint, save_checkpoint
from datasets.data_loader import Text2MelDataLoader

In [3]:
dataset_path = "datasets\TSP_M_Speakers\\"
tsp_speech = TSPSpeech([],"TSP_M_Speakers\\")

C:\Users\Karen 2\OneDrive\Documents\JHU\2020 Fall\Deep Learning\Final_project\DL-Project\pytorch-dc-tts\datasets\TSP_M_Speakers\


In [4]:
tsp_speech.fnames

['MA01_01',
 'MA01_02',
 'MA01_03',
 'MA01_04',
 'MA01_05',
 'MA01_06',
 'MA01_07',
 'MA01_08',
 'MA01_09',
 'MA01_10',
 'MA02_01',
 'MA02_02',
 'MA02_03',
 'MA02_04',
 'MA02_05',
 'MA02_06',
 'MA02_07',
 'MA02_08',
 'MA02_09',
 'MA02_10',
 'MA03_01',
 'MA03_02',
 'MA03_03',
 'MA03_04',
 'MA03_05',
 'MA03_06',
 'MA03_07',
 'MA03_08',
 'MA03_09',
 'MA03_10',
 'MA04_01',
 'MA04_02',
 'MA04_03',
 'MA04_04',
 'MA04_05',
 'MA04_06',
 'MA04_07',
 'MA04_08',
 'MA04_09',
 'MA04_10',
 'MA05_01',
 'MA05_02',
 'MA05_03',
 'MA05_04',
 'MA05_05',
 'MA05_06',
 'MA05_07',
 'MA05_08',
 'MA05_09',
 'MA05_10',
 'MA06_01',
 'MA06_02',
 'MA06_03',
 'MA06_04',
 'MA06_05',
 'MA06_06',
 'MA06_07',
 'MA06_08',
 'MA06_09',
 'MA06_10',
 'MB07_01',
 'MB07_02',
 'MB07_03',
 'MB07_04',
 'MB07_05',
 'MB07_06',
 'MB07_07',
 'MB07_08',
 'MB07_09',
 'MB07_10',
 'MB08_01',
 'MB08_02',
 'MB08_03',
 'MB08_04',
 'MB08_05',
 'MB08_06',
 'MB08_07',
 'MB08_08',
 'MB08_09',
 'MB08_10',
 'MB09_01',
 'MB09_02',
 'MB09_03',
 'MB

In [5]:
# Do preprocessing
preprocess(dataset_path, tsp_speech)

100%|████████████████████████████████████████████████████████████████████████████████| 660/660 [00:55<00:00, 11.86it/s]


In [ ]:
train_data_loader = Text2MelDataLoader(text2mel_dataset=TSPSpeech(['texts', 'mels', 'mel_gates']), batch_size=2,
                                       mode='train')

In [ ]:
next(iter(train_data_loader))

In [ ]:
TSPSpeech(['texts', 'mels', 'mel_gates'])

In [ ]:
testing = np.load("datasets/TSP_MD_Speaker/mels/MD19_01.npy")
import matplotlib.pyplot as plt
plt.imshow(testing.T, origin='lower')

In [ ]:
testing = np.load("datasets/LJSpeech-1.1/mels/LJ001-0001.npy")
import matplotlib.pyplot as plt
plt.imshow(testing.T, origin='lower')

In [ ]:
import os
import re
import codecs
import unicodedata
import numpy as np

vocab = "PE abcdefghijklmnopqrstuvwxyz'.?"  # P: Padding, E: EOS.
char2idx = {char: idx for idx, char in enumerate(vocab)}
idx2char = {idx: char for idx, char in enumerate(vocab)}


def text_normalize(text):
    text = ''.join(char for char in unicodedata.normalize('NFD', text)
                   if unicodedata.category(char) != 'Mn')  # Strip accents

    text = text.lower()
    text = re.sub("[^{}]".format(vocab), " ", text)
    text = re.sub("[ ]+", " ", text)
    return text


def read_metadata(metadata_file):
    fnames, text_lengths, texts = [], [], []
    print(fnames)
    transcript = os.path.join(metadata_file)
    lines = codecs.open(transcript, 'r', 'utf-8-sig').readlines()
    for line in lines:
        fname, _, text = line.strip().split("|")
        print(fname)
        fnames.append(fname)
        print(fnames)
        text = text_normalize(text) + "E"  # E: EOS
        text = [char2idx[char] for char in text]
        text_lengths.append(len(text))
        texts.append(np.array(text, np.long))
    print(fnames, text_lengths, texts, sep="\n")
    return fnames, text_lengths, texts


def get_test_data(sentences, max_n):
    normalized_sentences = [text_normalize(line).strip() + "E" for line in sentences]  # text normalization, E: EOS
    texts = np.zeros((len(normalized_sentences), max_n + 1), np.long)
    for i, sent in enumerate(normalized_sentences):
        texts[i, :len(sent)] = [char2idx[char] for char in sent]
    return texts


In [ ]:
os.getcwd()

In [ ]:
temp = read_metadata('datasets/TSP_M_Speakers/TSP-M_metadata.csv')